In [1]:
!pip install pandas numpy scikit-learn streamlit joblib matplotlib seaborn


In [2]:
import os

# Define folder names
folders = [
    "data",
    "data/raw",
    "data/processed",
    "models"
]

for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("Folders created/verified:")
for folder in folders:
    print(" -", folder)



Folders created/verified:
 - data
 - data/raw
 - data/processed
 - models


In [3]:
import pandas as pd

# Path to the raw data
data_path = "data/raw/streeteasy_listings.csv"

# Load data
df = pd.read_csv(data_path)

print("Shape of dataset:", df.shape)
df.head()


Shape of dataset: (3539, 18)


,rental_id,rent,bedrooms,bathrooms,size_sqft,min_to_subway,floor,building_age_yrs,no_fee,has_roofdeck,has_washer_dryer,has_doorman,has_elevator,has_dishwasher,has_patio,has_gym,neighborhood,borough
0,1545,2550,0.0,1,480,9,2.0,17,1,1,0,0,1,1,0,1,Upper East Side,Manhattan
1,2472,11500,2.0,2,2000,4,1.0,96,0,0,0,0,0,0,0,0,Greenwich Village,Manhattan
2,2919,4500,1.0,1,916,2,51.0,29,0,1,0,1,1,1,0,0,Midtown,Manhattan
3,2790,4795,1.0,1,975,3,8.0,31,0,0,0,1,1,1,0,1,Greenwich Village,Manhattan
4,3946,17500,2.0,2,4800,3,4.0,136,0,0,0,1,1,1,0,1,Soho,Manhattan


In [4]:
# Keep only the columns we care about
feature_cols = [
    "bedrooms",
    "bathrooms",
    "size_sqft",
    "min_to_subway",
    "floor",
    "building_age_yrs",
    "no_fee",
    "has_roofdeck",
    "has_washer_dryer",
    "has_doorman",
    "has_elevator",
    "has_dishwasher",
    "has_patio",
    "has_gym",
    "neighborhood",
    "borough",
]

target_col = "rent"

# Filter the dataframe
df_model = df[feature_cols + [target_col]].copy()

print("New shape after selecting features + target:", df_model.shape)
df_model.head()


New shape after selecting features + target: (3539, 17)


,bedrooms,bathrooms,size_sqft,min_to_subway,floor,building_age_yrs,no_fee,has_roofdeck,has_washer_dryer,has_doorman,has_elevator,has_dishwasher,has_patio,has_gym,neighborhood,borough,rent
0,0.0,1,480,9,2.0,17,1,1,0,0,1,1,0,1,Upper East Side,Manhattan,2550
1,2.0,2,2000,4,1.0,96,0,0,0,0,0,0,0,0,Greenwich Village,Manhattan,11500
2,1.0,1,916,2,51.0,29,0,1,0,1,1,1,0,0,Midtown,Manhattan,4500
3,1.0,1,975,3,8.0,31,0,0,0,1,1,1,0,1,Greenwich Village,Manhattan,4795
4,2.0,2,4800,3,4.0,136,0,0,0,1,1,1,0,1,Soho,Manhattan,17500


In [5]:
# Drop rows where target (rent) is missing
df_model = df_model.dropna(subset=[target_col])

# Quick summary of missing values in features
print("Missing values per column:")
print(df_model.isna().sum())


Missing values per column:
bedrooms            0
bathrooms           0
size_sqft           0
min_to_subway       0
floor               0
building_age_yrs    0
no_fee              0
has_roofdeck        0
has_washer_dryer    0
has_doorman         0
has_elevator        0
has_dishwasher      0
has_patio           0
has_gym             0
neighborhood        0
borough             0
rent                0
dtype: int64


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import joblib


In [7]:
# Separate X and y
X = df_model[feature_cols]
y = df_model[target_col]

# Identify numeric and categorical features
numeric_features = [
    "bedrooms",
    "bathrooms",
    "size_sqft",
    "min_to_subway",
    "floor",
    "building_age_yrs",
    "no_fee",
    "has_roofdeck",
    "has_washer_dryer",
    "has_doorman",
    "has_elevator",
    "has_dishwasher",
    "has_patio",
    "has_gym",
]

categorical_features = [
    "neighborhood",
    "borough",
]

# Preprocessing for numeric data
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

# Combine into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# Define the model
model = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

# Full pipeline: preprocessing + model
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", model)
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training shape:", X_train.shape)
print("Test shape:", X_test.shape)

# Fit the pipeline
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)

# Older sklearn: mean_squared_error has no 'squared' argument
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5   # take square root to get RMSE

r2 = r2_score(y_test, y_pred)

print("Model performance on test set:")
print(f"MAE  : {mae:,.2f} USD")
print(f"RMSE : {rmse:,.2f} USD")
print(f"R^2  : {r2:.3f}")



Training shape: (2831, 16)
Test shape: (708, 16)
Model performance on test set:
MAE  : 717.32 USD
RMSE : 1,335.45 USD
R^2  : 0.811


In [8]:
model_path = "models/rent_model.joblib"
joblib.dump(clf, model_path)

print(f"Model saved to: {model_path}")


Model saved to: models/rent_model.joblib


In [9]:
# Group by borough and neighborhood
neighborhood_stats = (
    df_model
    .groupby(["borough", "neighborhood"])
    .agg(
        listings_count=("rent", "count"),
        median_rent=("rent", "median"),
        mean_rent=("rent", "mean"),
        min_rent=("rent", "min"),
        max_rent=("rent", "max")
    )
    .reset_index()
)

print("Neighborhood stats shape:", neighborhood_stats.shape)
neighborhood_stats.head()


Neighborhood stats shape: (32, 7)


,borough,neighborhood,listings_count,median_rent,mean_rent,min_rent,max_rent
0,Manhattan,Battery Park City,104,5127.5,5968.961538,2600,13800
1,Manhattan,Central Harlem,82,2500.0,2963.926829,1425,7495
2,Manhattan,Central Park South,23,6995.0,7506.956522,2600,16000
3,Manhattan,Chelsea,182,5000.0,6193.483516,2495,16500
4,Manhattan,Chinatown,8,2900.0,2955.625000,1795,4100


In [10]:
processed_path = "data/processed/neighborhood_stats.csv"
neighborhood_stats.to_csv(processed_path, index=False)

print(f"Neighborhood stats saved to: {processed_path}")


Neighborhood stats saved to: data/processed/neighborhood_stats.csv


In [25]:
import os

print("Current working directory:")
print(os.getcwd())
print("\nFiles here:")
print(os.listdir())

# Check if rent_app.py is here
print("\nDoes rent_app.py exist here? ->", "rent_app.py" in os.listdir())

# Check if 'models' folder exists
print("\nDoes 'models' folder exist here? ->", "models" in os.listdir())


Current working directory:
C:\Users\KIIT0001\PROJECTS\NYC_Rent_Analyzer

Files here:
['.ipynb_checkpoints', 'analyzer_code.ipynb', 'data', 'models', 'rent_app.py', 'test.ipynb']

Does rent_app.py exist here? -> True

Does 'models' folder exist here? -> True


In [26]:
import os
import pandas as pd

# Ensure data folder exists
os.makedirs("data", exist_ok=True)

# Check raw dataset exists
RAW_PATH = "data/raw/streeteasy_listings.csv"

if not os.path.exists(RAW_PATH):
    raise FileNotFoundError(
        f"MISSING RAW DATA FILE: {RAW_PATH}\n\n"
        "Put your NYC rentals CSV inside the data folder and name it:\n"
        "streeteasy_rentals.csv"
    )

print("✅ Raw dataset found")

# Load raw data
df = pd.read_csv(RAW_PATH)

# Basic cleaning
df_clean = df[
    ["rent","bedrooms","bathrooms","size_sqft","borough","neighborhood"]
].dropna()

df_clean = df_clean[
    (df_clean["rent"] > 500) &
    (df_clean["rent"] < 20000) &
    (df_clean["bathrooms"] > 0) &
    (df_clean["size_sqft"] > 0)
]

# Save cleaned dataset
CLEAN_PATH = "data/processed/clean_listings.csv"
df_clean.to_csv(CLEAN_PATH, index=False)

print("✅ Clean dataset SAVED at:")
print(CLEAN_PATH)
print("Rows:", len(df_clean))


✅ Raw dataset found
✅ Clean dataset SAVED at:
data/processed/clean_listings.csv
Rows: 3522


In [27]:
!pip install shap statsmodels


   ---------------------------------------- 0.0/549.1 kB ? eta -:--:--
   ---------------------------------------- 549.1/549.1 kB 4.6 MB/s eta 0:00:00

   -------------------- ------------------- 1/2 [shap]
   -------------------- ------------------- 1/2 [shap]
   -------------------- ------------------- 1/2 [shap]
   -------------------- ------------------- 1/2 [shap]
   -------------------- ------------------- 1/2 [shap]
   -------------------- ------------------- 1/2 [shap]
   -------------------- ------------------- 1/2 [shap]
   -------------------- ------------------- 1/2 [shap]
   -------------------- ------------------- 1/2 [shap]
   ---------------------------------------- 2/2 [shap]



In [30]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

os.makedirs("data", exist_ok=True)
os.makedirs("models", exist_ok=True)

RAW_PATH = "data/raw/streeteasy_listings.csv"  # change if your file has another name

if not os.path.exists(RAW_PATH):
    raise FileNotFoundError(
        f"Dataset not found at {RAW_PATH}. "
        "Put your rentals CSV in 'data' and rename it to 'streeteasy_rentals.csv' or update RAW_PATH."
    )

df = pd.read_csv(RAW_PATH)
print("Raw shape:", df.shape)

# ------------------ define feature columns ------------------
FEATURE_COLS = [
    "bedrooms", "bathrooms", "size_sqft",
    "borough", "neighborhood",
    "no_fee", "has_dishwasher", "min_to_subway", "floor",
    "has_doorman", "building_age_yrs", "has_elevator", "has_patio",
    "has_roofdeck", "has_washer_dryer", "has_gym",
]

target_col = "rent"

# check columns
missing = [c for c in [target_col] + FEATURE_COLS if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns in CSV: {missing}")

# ------------------ build cleaned dataframe ------------------
cols_to_keep = [target_col] + FEATURE_COLS
if "created" in df.columns:
    cols_to_keep.append("created")   # keep for forecasting

df_clean = df[cols_to_keep].copy()

# drop rows with missing mandatory values
df_clean = df_clean.dropna(subset=[target_col, "bedrooms", "bathrooms", "size_sqft", "borough", "neighborhood"])

# basic filters
df_clean = df_clean[df_clean["rent"] > 500]
df_clean = df_clean[df_clean["rent"] < 20000]
df_clean = df_clean[df_clean["bathrooms"] > 0]
df_clean = df_clean[df_clean["size_sqft"] > 0]

print("Cleaned shape:", df_clean.shape)

# save cleaned data
clean_path = "data/processed/clean_listings.csv"
df_clean.to_csv(clean_path, index=False)
print(f"✅ Saved cleaned data to {clean_path}")

# ------------------ prepare for training ------------------
X = df_clean[FEATURE_COLS]
y = df_clean[target_col]

numeric_features = ["bedrooms", "bathrooms", "size_sqft",
                    "min_to_subway", "floor", "building_age_yrs"]

binary_features = [
    "no_fee", "has_dishwasher", "has_doorman",
    "has_elevator", "has_patio", "has_roofdeck",
    "has_washer_dryer", "has_gym",
]

categorical_features = ["borough", "neighborhood"]

numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

binary_transformer = "passthrough"  # already 0/1
categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("bin", binary_transformer, binary_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

model = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

regressor = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", model)
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train:", X_train.shape, "Test:", X_test.shape)

regressor.fit(X_train, y_train)
print("✅ Model trained.")

y_pred = regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)

# older sklearn doesn't support squared=False, so compute RMSE manually
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

r2 = r2_score(y_test, y_pred)

print("\nPerformance on test set:")
print("MAE :", round(mae, 2))
print("RMSE:", round(rmse, 2))
print("R²  :", round(r2, 3))

MODEL_PATH = "models/rent_model.pkl"
joblib.dump(regressor, MODEL_PATH)
print(f"\n✅ Saved model to {MODEL_PATH}")


Raw shape: (3539, 18)
Cleaned shape: (3522, 17)
✅ Saved cleaned data to data/processed/clean_listings.csv
Train: (2817, 16) Test: (705, 16)
✅ Model trained.

Performance on test set:
MAE : 703.7
RMSE: 1261.82
R²  : 0.839

✅ Saved model to models/rent_model.pkl


In [36]:
%%writefile rent_app.py
import streamlit as st
import pandas as pd
import joblib
import os

# ---------- Paths ----------
MODEL_PATH = "models/rent_model.pkl"
CLEAN_DATA_PATH = "data/processed/clean_listings.csv"

# ---------- Loaders ----------
@st.cache_resource
def load_model():
    if not os.path.exists(MODEL_PATH):
        st.error(f"Model file not found at {MODEL_PATH}. Run the training notebook first.")
        st.stop()
    return joblib.load(MODEL_PATH)

@st.cache_data
def load_clean_data():
    if not os.path.exists(CLEAN_DATA_PATH):
        st.error(f"Clean data file not found at {CLEAN_DATA_PATH}. Run the training notebook first.")
        st.stop()
    return pd.read_csv(CLEAN_DATA_PATH)

# ---------- Main ----------
def main():
    st.set_page_config(page_title="NYC Rent Analyzer", page_icon="🏙️", layout="wide")

    st.title("🏙️ NYC Rent Analyzer")
    st.write("Estimate fair rent prices using a Machine Learning model trained on NYC rentals.")

    model = load_model()
    df_clean = load_clean_data()

    st.header("💰 Price Estimator")

    # Dropdowns from data
    boroughs = sorted(df_clean["borough"].dropna().unique().tolist())
    borough = st.selectbox("Borough", boroughs)

    neighborhoods = sorted(
        df_clean.loc[df_clean["borough"] == borough, "neighborhood"].dropna().unique().tolist()
    )
    neighborhood = st.selectbox("Neighborhood", neighborhoods)

    col1, col2, col3 = st.columns(3)
    with col1:
        bedrooms = st.number_input("Bedrooms", min_value=0, max_value=10, value=1, step=1)
    with col2:
        bathrooms = st.number_input("Bathrooms", min_value=1.0, max_value=5.0, value=1.0, step=0.5)
    with col3:
        size_sqft = st.number_input("Size (sqft)", min_value=100, max_value=5000, value=600, step=50)

    st.markdown("---")
    st.subheader("🏢 Building & Amenities (used in your trained model)")

    colA, colB, colC = st.columns(3)

    with colA:
        floor = st.number_input("Floor", min_value=0, max_value=80, value=3, step=1)
        building_age_yrs = st.number_input("Building age (years)", min_value=0, max_value=200, value=20, step=1)
        min_to_subway = st.number_input("Minutes to subway", min_value=0, max_value=60, value=5, step=1)

    with colB:
        no_fee = st.checkbox("No broker fee")
        has_doorman = st.checkbox("Doorman")
        has_elevator = st.checkbox("Elevator")
        has_gym = st.checkbox("Gym")

    with colC:
        has_dishwasher = st.checkbox("Dishwasher")
        has_patio = st.checkbox("Patio/Balcony")
        has_roofdeck = st.checkbox("Roof deck")
        has_washer_dryer = st.checkbox("Washer/Dryer in unit")

    listed_rent = st.number_input(
        "Optional: Listed Rent (USD)",
        min_value=0, max_value=20000, value=0, step=50,
        help="If you enter the asking rent, I will tell you if it's over/underpriced."
    )

    if st.button("Predict Rent"):
        # Build input with ALL columns your model expects
        input_dict = {
            "bedrooms": [bedrooms],
            "bathrooms": [bathrooms],
            "size_sqft": [size_sqft],
            "borough": [borough],
            "neighborhood": [neighborhood],
            "no_fee": [int(no_fee)],
            "has_dishwasher": [int(has_dishwasher)],
            "min_to_subway": [min_to_subway],
            "floor": [floor],
            "has_doorman": [int(has_doorman)],
            "building_age_yrs": [building_age_yrs],
            "has_elevator": [int(has_elevator)],
            "has_patio": [int(has_patio)],
            "has_roofdeck": [int(has_roofdeck)],
            "has_washer_dryer": [int(has_washer_dryer)],
            "has_gym": [int(has_gym)],
        }

        input_df = pd.DataFrame(input_dict)

        # DEBUG (optional): show the row being sent to the model
        # st.write("Input sent to model:", input_df)

        predicted = model.predict(input_df)[0]

        st.subheader("Prediction")
        st.write(f"**Estimated fair rent:** ${predicted:,.0f} / month")

        if listed_rent > 0:
            diff = listed_rent - predicted
            diff_pct = diff / predicted * 100

            if diff_pct > 15:
                label = "🚨 Overpriced"
            elif diff_pct < -15:
                label = "✅ Underpriced (Good deal)"
            else:
                label = "⚖️ Fairly priced"

            st.markdown(
                f"{label} – listed rent is ${listed_rent:,.0f}, which is {diff_pct:+.1f}% "
                f"relative to the model estimate."
            )

            progress_value = int(50 + diff_pct)
            progress_value = max(0, min(100, progress_value))
            st.progress(progress_value)

if __name__ == "__main__":
    main()


Overwriting rent_app.py
